In [99]:
# ! pip install nbformat>=4.2.0

In [100]:
import os, sys
from datetime import datetime as dt, timedelta as td
from time import sleep
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


absolute_parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(absolute_parent)

from tdf_utility.trading.nse_api import NSE_API

nse_api = NSE_API()

In [ ]:
top_10_stock_url = "https://www.nseindia.com/api/NextApi/apiClient/GetQuoteApi?functionName=getTopTenStock"
nifty_index = "https://www.nseindia.com/api/NextApi/apiClient?functionName=getIndexData&&type=All"

In [123]:
safe_symbol = "EICHERMOT"
to_dt = dt.now()
from_dt = to_dt - td(days=90)

print(safe_symbol, from_dt, to_dt)

EICHERMOT 2025-10-21 01:09:54.414987 2026-01-19 01:09:54.414987


In [124]:
company_historical_tradedata_url = f'api/NextApi/apiClient/GetQuoteApi?functionName=getHistoricalTradeData&symbol={safe_symbol}&series=EQ&fromDate={from_dt.strftime("%d-%m-%Y")}&toDate={to_dt.strftime("%d-%m-%Y")}'
company_trade_data = nse_api._get_data(company_historical_tradedata_url)
company_trade_data_df = pd.DataFrame(company_trade_data)
company_trade_data_df["Date"] = pd.to_datetime(company_trade_data_df.get("mtimestamp"), format='%d-%b-%Y')

company_trade_data_df.rename(columns={'chSymbol': 'Symbol', 'chSeries': 'Series', 'chPreviousClsPrice': 'Prev Close', 
        'chOpeningPrice': 'Open', 'chTradeHighPrice': 'High', 'chTradeLowPrice': 'Low', 'chLastTradedPrice': 'LTP',
       'chClosingPrice': 'Close', 'vwap': 'VWAP', 'chTotTradedQty': 'Volumn', 'chTotTradedVal': 'Traded Value',
       'chTotalTrades': 'Transactions', 'ch52WeekHighPrice': '52Week High', 'ch52WeekLowPrice': '52Week Low'}, inplace=True)
company_trade_data_df = company_trade_data_df.sort_values(by='Date', ascending=False)
company_trade_data_df.reset_index(drop=True, inplace=True)

In [125]:
monthly_expiry_dates_url = f"api/NextApi/apiClient/GetQuoteApi?functionName=getStrPriceExpiryBySymbol&symbol={safe_symbol}&fromDate={from_dt.strftime("%d-%m-%Y")}&toDate={to_dt.strftime("%d-%m-%Y")}"
monthly_expiry_dates_df = pd.DataFrame(nse_api._get_data(monthly_expiry_dates_url), columns=["Expiry Date"])
monthly_expiry_dates_df["Expiry Date"] = pd.to_datetime(monthly_expiry_dates_df["Expiry Date"], format='%d-%b-%Y').dt.date
monthly_expiry_dates_df["From Date"] = monthly_expiry_dates_df["Expiry Date"].shift(1) + td(days=1)
monthly_expiry_dates_df["To Date"] = monthly_expiry_dates_df["Expiry Date"]
monthly_expiry_dates_df.dropna(inplace=True)
current_date = pd.Timestamp.now().date()
monthly_expiry_dates_df = monthly_expiry_dates_df[monthly_expiry_dates_df["From Date"] <= current_date]
monthly_expiry_dates_df["To Date"] = monthly_expiry_dates_df["To Date"].apply(lambda x: min(x, current_date) if x > current_date else x)

In [126]:
company_trade_data_df

,Symbol,Series,Prev Close,Open,High,Low,LTP,Close,VWAP,Volumn,Traded Value,Transactions,52Week High,52Week Low,mtimestamp,Date
0,EICHERMOT,EQ,7358.0,7330.0,7417.0,7298.5,7300.0,7315.0,7343.85,441023,3.238806e+09,64247,7613.5,4646.00,16-Jan-2026,2026-01-16
1,EICHERMOT,EQ,7404.0,7390.0,7390.0,7236.0,7370.0,7358.0,7313.34,426082,3.116081e+09,66658,7613.5,4646.00,14-Jan-2026,2026-01-14
2,EICHERMOT,EQ,7436.0,7488.0,7514.5,7348.0,7383.0,7404.0,7418.59,262636,1.948389e+09,36825,7613.5,4646.00,13-Jan-2026,2026-01-13
3,EICHERMOT,EQ,7507.0,7477.0,7507.0,7322.0,7443.0,7436.0,7402.51,315579,2.336077e+09,48150,7613.5,4646.00,12-Jan-2026,2026-01-12
4,EICHERMOT,EQ,7551.0,7582.0,7613.5,7486.0,7510.0,7507.0,7560.26,474627,3.588304e+09,50025,7613.5,4646.00,09-Jan-2026,2026-01-09
5,EICHERMOT,EQ,7582.5,7590.0,7601.0,7539.0,7552.5,7551.0,7565.52,459727,3.478075e+09,44411,7601.0,4646.00,08-Jan-2026,2026-01-08
6,EICHERMOT,EQ,7522.5,7523.0,7597.0,7490.0,7597.0,7582.5,7557.12,380761,2.877456e+09,56894,7597.0,4646.00,07-Jan-2026,2026-01-07
7,EICHERMOT,EQ,7482.5,7514.0,7569.0,7484.5,7524.0,7522.5,7534.96,396099,2.984590e+09,46827,7569.0,4646.00,06-Jan-2026,2026-01-06
8,EICHERMOT,EQ,7334.5,7344.0,7514.5,7328.0,7494.0,7482.5,7465.96,426798,3.186457e+09,43626,7514.5,4646.00,05-Jan-2026,2026-01-05
9,EICHERMOT,EQ,7348.0,7348.0,7382.5,7298.0,7344.0,7334.5,7341.78,227469,1.670026e+09,46358,7382.5,4646.00,02-Jan-2026,2026-01-02


In [127]:
monthly_expiry_dates_df

,Expiry Date,From Date,To Date
1,2025-11-25,2025-10-29,2025-11-25
2,2025-12-30,2025-11-26,2025-12-30
3,2026-01-27,2025-12-31,2026-01-19


In [ ]:
def generate_custom_monthly_df(daily_df, expiry_schedule_df):
    monthly_records = []
    # 2. Loop through each row in the custom expiry schedule
    for _, row in expiry_schedule_df.iterrows():
        # start_date = row['From Date']
        # end_date = row['To Date']
        start_date = pd.to_datetime(row['From Date'])
        end_date = pd.to_datetime(row['To Date'])
        
        # 3. Filter daily data for this specific date range
        # Logic: Date must be >= start AND <= end
        mask = (daily_df['Date'] >= start_date) & (daily_df['Date'] <= end_date)
        period_data = daily_df.loc[mask]
        
        # If no trading data exists for this period, skip it
        if period_data.empty:
            continue
            
        # 4. Calculate Candle Values
        monthly_candle = {
            "Expiry Date": row['Expiry Date'],
            "From Date": start_date,  # The official closing date of this candle
            "To Date": end_date,
            "Open": period_data.iloc[0]['Open'],       # Open price of the FIRST day
            "High": period_data['High'].max(),         # Highest price seen in the period
            "Low": period_data['Low'].min(),           # Lowest price seen in the period
            "Close": period_data.iloc[-1]['Close'],    # Close price of the LAST day
            # "Volume": period_data['Volume'].sum()      # Total volume traded
            "52Week High": period_data['52Week High'].max(), 
            # "52Week High Date": period_data['52Week High Date'].max(), 
            "52Week Low": period_data['52Week Low'].max(), 
            # "52Week Low Date": period_data['52Week Low Date'].max(), 
        }
        monthly_records.append(monthly_candle)
    # 5. Convert list of dicts to a clean DataFrame
    return pd.DataFrame(monthly_records)

In [138]:
out = generate_custom_monthly_df(daily_df=company_trade_data_df, expiry_schedule_df=monthly_expiry_dates_df)

In [139]:
out

,Expiry Date,From Date,To Date,Open,High,Low,Close,52Week High,52Week Low
0,2025-11-25,2025-10-29,2025-11-25,7258.5,7284.5,6670.0,6952.0,7284.5,4646.0
1,2025-12-30,2025-11-26,2025-12-30,7272.0,7374.5,6956.0,7198.5,7374.5,4646.0
2,2026-01-27,2025-12-31,2026-01-19,7330.0,7613.5,7140.0,7312.5,7613.5,4646.0


1. Rapid Api
2. Apify
3. apidog
4. https://developer.schwab.com/user-guides/get-started/introduction
